# 初始化

In [1]:
%cd '/home/xiaobo/Project/Trans-WSSS'
%pwd

import sys

import cv2
import matplotlib.pyplot as plt

sys.path.append("others/segment_anything")
from segment_anything import sam_model_registry

sys.path = ['.', './src'] + sys.path
from libs.sam import SamAuto
from libs.viz.viz_anns import show_imgs_anns

/home/xiaobo/Project/Trans-WSSS


/home/xiaobo/anaconda3/envs/p10-timm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 读取图片和SAM。

In [2]:
img_ids = ['2007_000032', '2007_000170', '2007_003205',
           '2007_009724', '2008_001784', '2008_006844',
           '2008_007281', '2010_004844', '2009_004050']

imgs = [cv2.cvtColor(cv2.imread(f'datasets/VOC2012/JPEGImages/{img_id}.jpg'), cv2.COLOR_BGR2RGB) for img_id in img_ids]

In [3]:
sam_checkpoint = "pretrains/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda:7"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam = sam.to(device=device)

# 建立SAM自动分割器，并推理。

In [4]:
# * 官方Jupyter V2
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
# )
# * 默认值.
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.88,
#     stability_score_thresh=0.95,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=0,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=1,
#     min_mask_region_area=0,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
# )
# * SSA Light
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=16,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     crop_n_layers=0,  # 1 by default
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode='binary_mask',
# )
# * SSA Light 8点
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=8,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     crop_n_layers=0,  # 1 by default
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode='binary_mask',
# )
# * SSA Default
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=32,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     crop_n_layers=0,  # 1 by default
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode='binary_mask',
# )
# * SSA Heavy
# mask_generator = SamAutomaticMaskGenerator(
#     model=sam,
#     points_per_side=64,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     crop_n_layers=1,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode='binary_mask',
# )
# * SAM Auto阈值滤除分分、分。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(-1.0, -1.0, 0.0),
#     score_nms_offset=(0.0, 0.0, 0.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * SAM Auto阈值滤除分分、分，并提高总级的阈值得分偏移。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(-1.0, -1.0, 0.1),
#     score_nms_offset=(0.0, 0.0, 0.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * 降低总的稳定度阈值。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(-1.0, -1.0, 0.1),
#     score_nms_offset=(0.0, 0.0, 0.0),
#     stability_score_bias=(0.0, 0.0, 0.04),
# )
# * SAM Auto总NMS优先。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(0.0, 0.0, 0.0),
#     score_nms_offset=(0.0, 0.0, 1.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * SAM Auto总NMS优先，提高总级的阈值得分偏移。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(0.0, 0.0, 0.1),
#     score_nms_offset=(0.0, 0.0, 1.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * SAM Auto总NMS优先，提高总级的阈值得分偏移，降低NMS阈值。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.3,
#     crop_n_layers=1,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(0.0, 0.0, 0.1),
#     score_nms_offset=(0.0, 0.0, 1.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * SAM Auto总NMS优先，提高总级的阈值得分偏移，crop0次。
# mask_generator = SamAuto(
#     model=sam,
#     points_per_side=32,
#     points_per_batch=256,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     stability_score_offset=1.0,
#     box_nms_thresh=0.7,
#     crop_n_layers=0,
#     crop_nms_thresh=0.7,
#     crop_overlap_ratio=512 / 1500,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
#     output_mode="binary_mask",
#     score_thresh_offset=(0.0, 0.0, 0.1),
#     score_nms_offset=(0.0, 0.0, 1.0),
#     stability_score_bias=(0.0, 0.0, 0.0),
# )
# * SAM Auto总NMS优先，提高总级的阈值得分偏移，结合石沙水过滤。
mask_generator = SamAuto(
    model=sam,
    points_per_side=32,
    points_per_batch=256,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    stability_score_offset=1.0,
    box_nms_thresh=0.7,
    crop_n_layers=1,
    crop_nms_thresh=0.7,
    crop_overlap_ratio=512 / 1500,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
    output_mode="binary_mask",
    score_thresh_offset=(0.0, 0.0, 0.1),
    score_nms_offset=(0.0, 0.0, 1.0),
    stability_score_bias=(0.0, 0.0, 0.0),
    rock_sand_water_thresh=0.3,
    rock_sand_water_chunk_size=50,
)

In [5]:
img_masks = [mask_generator.generate(img) for img in imgs]

In [ ]:
fig: plt.Figure = plt.figure(dpi=200)
show_imgs_anns(fig, imgs, img_ids, img_masks, levels=(0, 1, 2))
fig.show()

fig: plt.Figure = plt.figure(dpi=200)
show_imgs_anns(fig, imgs, img_ids, img_masks, levels=(0, 1))
fig.show()

fig: plt.Figure = plt.figure(dpi=200)
show_imgs_anns(fig, imgs, img_ids, img_masks, levels=(2,))
fig.show()